In [1]:
import re
import string
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
def retrieve_docs_and_clean():

  r = requests.get('https://sports.ndtv.com/fifa-world-cup-2022/news')
  soup = BeautifulSoup(r.content, 'html.parser')

  #THE FOLLOWING CODE NEED TO BE MODIFIED TO SUITE FOR THE ABOVE URL
  link = []
  for i in soup.find('div', {'class':'lst-pg_hd'}).find_all('a',{'class':'lst-pg_ttl'}):
      i['href'] ='https://sports.ndtv.com/'+ i['href'] + '?page=all'
      link.append(i['href'])
  

  # Retrieve Paragraphs
  documents = []
  for i in link:
      r = requests.get(i)
      soup = BeautifulSoup(r.content, 'html.parser')

      sen = []
      for i in soup.find('div', {'class':'sp-cn pg-str-com js-ad-section'}).find_all('p'):
          sen.append(i.text)
      documents.append(' '.join(sen))
  # Clean Paragraphs
  documents_clean = []
  for d in documents:
      document_test = re.sub(r'[^\x00-\x7F]+', ' ', d)
      document_test = re.sub(r'@\w+', '', document_test)
      document_test = document_test.lower()
      document_test = re.sub(r'[%s]' % re.escape(string.punctuation), ' ', document_test)
      document_test = re.sub(r'[0-9]', '', document_test)
      document_test = re.sub(r'\s{2,}', ' ', document_test)
      documents_clean.append(document_test)

  return documents_clean

In [3]:
docs = retrieve_docs_and_clean()
# Create Term-Document Matrix with TF-IDF weighting
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(docs)

# Create a DataFrame
df = pd.DataFrame(X.T.toarray(), index=vectorizer.get_feature_names_out())
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
abandoned,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.096107
abemahttps,0.020915,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.000000
ability,0.000000,0.0,0.076002,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.06437,0.000000,0.000000,0.0,0.0,0.0,0.000000
about,0.000000,0.0,0.046770,0.019402,0.0,0.0,0.0,0.0,0.0,0.0,0.053668,0.00000,0.051215,0.000000,0.0,0.0,0.0,0.000000
above,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.000000,0.024622,0.0,0.0,0.0,0.000000


In [10]:
def get_similar_articles(q, df):
  print("query:", q)
  print("The following are articles with the highest cosine similarity values: ")
  print()
  q = [q]
  q_vec = vectorizer.transform(q).toarray().reshape(df.shape[0],)
  sim = {}
  for i in range(10):
    sim[i] = np.dot(df.loc[:, i].values, q_vec) / np.linalg.norm(df.loc[:, i]) * np.linalg.norm(q_vec)
  
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  
  for k, v in sim_sorted:
    if v != 0.0:
      print("Similarity Values:", v)
      print(docs[k])
      print()


q1 = 'brazil'
q2 = 'england'
q3 = 'france'

get_similar_articles(q1, df)
print('-'*100)
get_similar_articles(q2, df)
print('-'*100)
get_similar_articles(q3, df)

query: brazil
The following are articles with the highest cosine similarity values: 

Similarity Values: 0.06234607566313829
neymar could make his return to the world cup stage on monday as brazil continue their bid to be crowned kings for a record extending sixth time against south korea superstar forward neymar has been absent for brazil since spraining his ankle in his team s opening group g win against serbia last month and their supporters have been sweating on his fitness ever since coach tite said neymar would be assessed in brazil s final pre game training session on sunday but gave a heavy hint that the paris saint germain attacker would start window rrcode window rrcode rrcode push function function v d o ai ai d createelement script ai defer true ai async true ai src v location protocol o d head appendchild ai window document a vdo ai core v ndtv vdo ai js he s going to train this afternoon and if he s ok he will play tomorrow tite told reporters his return would be a big bo

In [7]:
from gensim.summarization.bm25 import BM25

def simple_tok(sent:str):
    return sent.split()

def bm25_similar_articles(query):
  print("query:", query)
  print("The following are articles with the highest BM25 scores: ")
  print()
  tok_corpus = [simple_tok(s) for s in docs]
  query = simple_tok(query)
  bm25 = BM25(tok_corpus)
  scores = bm25.get_scores(query, average_idf = 100)
  best_docs = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:3]
  for i, b in enumerate(best_docs):
      print(f"rank {i+1}: {docs[b]}")
      print()

In [11]:
q1 = 'brazil'
q2 = 'england'
q3 = 'france'


bm25_similar_articles(q1)
print('-'*100)
bm25_similar_articles(q2)
print('-'*100)
bm25_similar_articles(q3)
print('-'*100)

query: brazil
The following are articles with the highest BM25 scores: 

rank 1: fans of football superstar pele congregated sunday outside the sao paulo hospital where the year old brazilian icon is being treated for a respiratory infection more than devotees prayed for the recovery of the man widely regarded as the greatest footballer of all time the three time world champion had been hospitalized on tuesday amid ongoing treatments for colon cancer a condition first diagnosed in september we are a spiritual force praying for the sporting idol as he wages one of the toughest battles of his life one fan marcos bispo dos santos told afp window rrcode window rrcode rrcode push function function v d o ai ai d createelement script ai defer true ai async true ai src v location protocol o d head appendchild ai window document a vdo ai core v ndtv vdo ai js doctors at sao paulo s albert einstein hospital said saturday that pele born edson arantes do nascimento remained stable pele has had a g